In [57]:
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import spotipy.util as util
import spotipy.oauth2 as oauth2
import pandas as pd
import numpy as np
import random
import time

In [58]:
def load_data(filename):
    return pd.read_csv(filename)

In [59]:
def setup_spotify(client_id='', 
                  client_secret=''):
    
    spotify = spotipy.Spotify()

    CLIENT_ID = client_id
    CLIENT_SECRET = client_secret

    credentials = oauth2.SpotifyClientCredentials(
            client_id=CLIENT_ID,
            client_secret=CLIENT_SECRET)

    token = credentials.get_access_token()
    spotify = spotipy.Spotify(auth=token)
    
    return spotify 

In [60]:
def recursive_spotify_searcher(spotify, track_uri):
    try:
        return [spotify.tracks(track_uri),
                spotify.audio_features(track_uri)]
    except Exception as e: 
        #print(e)
        print("Waiting 10 seconds...")
        time.sleep(10)
        return recursive_spotify_searcher(spotify, track_uri)

In [61]:
def get_audio_features(audio_features, data):
    
    for audio in audio_features:
        data[0].append(audio["uri"])
        data[1].append(audio["danceability"])
        data[2].append(audio["energy"])
        data[3].append(audio["key"])
        data[4].append(audio["loudness"])
        data[5].append(audio["mode"])
        data[6].append(audio["acousticness"])
        data[7].append(audio["instrumentalness"])
        data[8].append(audio["liveness"])
        data[9].append(audio["valence"])
        data[10].append(audio["tempo"])

In [62]:
def get_track_features(track_features, data):

    for track in track_features["tracks"]:
        if not track:
            data[11].append(-1)
            data[12].append(-1)
        else:
            data[11].append(track["popularity"])
            data[12].append(track["album"]["release_date"])    

In [63]:
spotify = setup_spotify()

In [64]:
data = []

for i in range(13):
    data.append([])

In [65]:
songs = load_data("./Songs/songs1.csv")

unique_songs = songs.groupby("track_uri").size().reset_index()
track_uri = unique_songs["track_uri"].str.split(':')

len(unique_songs)

36175

In [66]:
for i in range(0, len(unique_songs), 50):
    print(i)
    
    track_uri = unique_songs["track_uri"][i:i+50].values
    tracks = recursive_spotify_searcher(spotify, track_uri)
    get_track_features(tracks[0], data)
    get_audio_features(tracks[1], data)
    time.sleep(random.choice(np.arange(5)))

0
[['spotify:track:000xQL6tZNLJzIrtIgxqSl', 'spotify:track:002PgfoyfrOGiKch4EW8Wm', 'spotify:track:004skCQeDn1iLntSom0rRr', 'spotify:track:005CGalYNgMNZcvWMIFeK8', 'spotify:track:005X0FmdtkM1kiutosXLTR', 'spotify:track:009NFK4GeY7xzHCJPrPKdz', 'spotify:track:00AxNl4D4jHL2AEf1W55j5', 'spotify:track:00BnfL75e8vHSGCmwUWbEk', 'spotify:track:00BuKLSAFkaEkaVAgIMbeA', 'spotify:track:00CqEmnPLFKDhAb3cuu6Cs', 'spotify:track:00CujlY4ya5p2FNDu9BCZP', 'spotify:track:00EE6U2juZMvGuMUCTHuas', 'spotify:track:00EfcDboIDLWzvPypeWGDg', 'spotify:track:00FR9VQ0uzF4NNxVKKiMz2', 'spotify:track:00FROhC5g4iJdax5US8jRr', 'spotify:track:00FU5uypELwgQuM9IbFC6G', 'spotify:track:00FaBIFXUfzaaVcsQmTTJY', 'spotify:track:00FjRu5wKkgDfiHognphYv', 'spotify:track:00H4w1nxlzue3YNjaa3Kwg', 'spotify:track:00HZJG1FRNi2Q7KAkZ3UOv', 'spotify:track:00LfFm08VWeZwB0Zlm24AT', 'spotify:track:00NUqFMIpCsrYPbM9YpVHQ', 'spotify:track:00NmK2seXpRFNAEsRBe7lc', 'spotify:track:00NnZ4CDQ7opiVRImcPc9x', 'spotify:track:00OY7B2Tod8sXcZOaK2dI

In [50]:
columns = ["uri", "danceability", "energy", "key", "loudness",
           "mode", "acousticness", "instrumentalness", 
           "liveness", "valence", "tempo", "popularity", "release_date"]

df = {
    "uri": data[0],
    "danceability": data[1],
    "energy": data[2], 
    "key": data[3], 
    "loudness": data[4],
    "mode": data[5], 
    "acousticness": data[6], 
    "instrumentalness": data[7], 
    "liveness": data[8], 
    "valence": data[9], 
    "tempo": data[10],
    "popularity": data[11], 
    "release_date": data[12]
}

data = pd.DataFrame(df, columns=columns)

In [53]:
data.to_csv("./Songs_features/song1.csv", index=False)